https://gist.github.com/radcli14/1e5923cc3927debc755aada7f40eb22c

https://moorepants.github.io/learn-multibody-dynamics/configuration.html#calculating-additional-kinematic-quantities

In [413]:
import sympy as sp
import sympy.physics.mechanics as me

# sp.init_printing(use_latex='mathjax')
# me.init_vprinting()
me.mechanics_printing()

In [414]:
from IPython.display import display, Latex

def dtex(*args):
    tex = ''
    for arg in args:
        if isinstance(arg, str):
            # Handle string arguments
            tex += f'${arg}$ $\, = \,$ '
        else:
            # Handle SymPy arguments
            tex += f'${me.mlatex(arg)}$ $\, = \,$ '
    
    # Trim off the last equals sign
    tex = tex[:-10]  
    display(Latex(tex))

<>:8: SyntaxWarning:

invalid escape sequence '\,'

<>:11: SyntaxWarning:

invalid escape sequence '\,'

<>:8: SyntaxWarning:

invalid escape sequence '\,'

<>:11: SyntaxWarning:

invalid escape sequence '\,'

C:\Users\phcre\AppData\Local\Temp\ipykernel_25304\3903178680.py:8: SyntaxWarning:

invalid escape sequence '\,'

C:\Users\phcre\AppData\Local\Temp\ipykernel_25304\3903178680.py:11: SyntaxWarning:

invalid escape sequence '\,'



In [415]:
# Inertial Reference Frame
N = me.ReferenceFrame('N')
N.x, N.y, N.z

(n_x, n_y, n_z)

Frames

In [416]:
l_a, l_b, l_c, l_n = sp.symbols('l_a l_b l_c l_n')
q = theta_1, theta_2, theta_3 = me.dynamicsymbols('theta_1 theta_2 theta_3')
s = omega_1, omega_2, omega_3 = me.dynamicsymbols('omega_1, omega_2, omega_3')
t = me.dynamicsymbols._t

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
C = me.ReferenceFrame('C')
A.orient_axis(N, N.z, theta_1)
B.orient_axis(N, N.z, theta_2)
C.orient_axis(N, N.z, theta_3)
# C.orient_axis(B, B.z, me.dot(B.x, C.x)) # TODO: this might work as it constrains the rotation of C with respect to B

# A.set_ang_vel(N, omega_1*N.z)
# B.set_ang_vel(N, omega_2*N.z)
# C.set_ang_vel(N, omega_3*N.z)


print(f'Rotation matrix for the A frame')
dtex('\\mathbf{R}_A', A.dcm(N))
print(f'Angular rate for the A frame')
dtex('\\mathbf{\\omega}_A', A.ang_vel_in(N))

print(f'Rotation matrix for the B frame')
dtex('\\mathbf{R}_B', B.dcm(N))
print(f'Angular rate for the B frame')
dtex('\\mathbf{\\omega}_B', B.ang_vel_in(N))

print(f'Rotation matrix for the C frame')
dtex('\\mathbf{R}_C', sp.trigsimp(C.dcm(N)))
print(f'Angular rate for the C frame')
dtex('\\mathbf{\\omega}_C', C.ang_vel_in(N))

Rotation matrix for the A frame


<IPython.core.display.Latex object>

Angular rate for the A frame


<IPython.core.display.Latex object>

Rotation matrix for the B frame


...

<IPython.core.display.Latex object>

Rotation matrix for the C frame


<IPython.core.display.Latex object>

Angular rate for the C frame


<IPython.core.display.Latex object>

In [417]:
me.dot(B.x, C.x)

sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃)

In [418]:
sp.trigsimp(me.dot(B.x, C.x))

cos(θ₂ - θ₃)

Points

In [419]:
O = me.Point('O')
O.set_vel(N, 0)

P1 = me.Point('P1')
P2 = me.Point('P2')
P3 = me.Point('P3')
P4 = me.Point('P4')

P2.set_pos(P1, l_a*A.x)
P3.set_pos(P2, l_b*B.x)
P4.set_pos(P3, l_c*C.x)

# P4.pos_from(P1)
P2.pos_from(P1)

lₐ a_x

In [420]:
loop = P4.pos_from(P1) - l_n*N.x
display(loop)

l_c c_x + l_b b_x + lₐ a_x + -lₙ n_x

Constraint equations

In [421]:
fhx = sp.trigsimp(loop.dot(N.x))
fhy = sp.trigsimp(loop.dot(N.y))
fh = sp.Matrix([fhx, fhy, me.dot(B.x, C.x)])
fh = sp.trigsimp(fh) # warning, this simplification is slow and may not be necessary
fh

⎡lₐ⋅cos(θ₁) + l_b⋅cos(θ₂) + l_c⋅cos(θ₃) - lₙ⎤
⎢                                           ⎥
⎢  lₐ⋅sin(θ₁) + l_b⋅sin(θ₂) + l_c⋅sin(θ₃)   ⎥
⎢                                           ⎥
⎣               cos(θ₂ - θ₃)                ⎦

In [422]:
fhd = fh.diff(t)
fhd

⎡-lₐ⋅sin(θ₁)⋅θ₁̇ - l_b⋅sin(θ₂)⋅θ₂̇ - l_c⋅sin(θ₃)⋅θ₃̇⎤
⎢                                                ⎥
⎢lₐ⋅cos(θ₁)⋅θ₁̇ + l_b⋅cos(θ₂)⋅θ₂̇ + l_c⋅cos(θ₃)⋅θ₃̇ ⎥
⎢                                                ⎥
⎣            -(θ₂̇ - θ₃̇)⋅sin(θ₂ - θ₃)             ⎦

In [423]:
# Constraint equations
# fc = loop
# fv = fc.dt(N)

# Constraint matrix equations in inertial frame
# position_constraint = sp.Matrix(fc.to_matrix(N)[:2])
# velocity_constraint = sp.Matrix(fv.to_matrix(N)[:2])

print('Position constraint equation')
dtex(loop, '0')

print('Position constraint equations in inertial frame')
dtex(fh)

# print('Velocity constraint equation')
# dtex(fv, '0')

print('Velocity constraint equations in inertial frame')
dtex(fhd)

Position constraint equation


<IPython.core.display.Latex object>

Position constraint equations in inertial frame


<IPython.core.display.Latex object>

Velocity constraint equations in inertial frame


<IPython.core.display.Latex object>

In [424]:
me.find_dynamicsymbols(fh)

{θ₁, θ₂, θ₃}

In [425]:
import math  # provides pi as a float

repl = {
    l_a: 1.0,
    # l_b: unknown,
    l_c: 1.0,
    l_n: 4.0,
    theta_1: 30.0*math.pi/180.0,
    # theta_2: unknown,
    # theta_3: unknown,
}
repl

{lₐ: 1.0, l_c: 1.0, lₙ: 4.0, θ₁: 0.5235987755982988}

In [426]:
fh.xreplace(repl)

⎡l_b⋅cos(θ₂) + 1.0⋅cos(θ₃) - 3.13397459621556⎤
⎢                                            ⎥
⎢      l_b⋅sin(θ₂) + 1.0⋅sin(θ₃) + 0.5       ⎥
⎢                                            ⎥
⎣                cos(θ₂ - θ₃)                ⎦

In [427]:
theta_2_guess = 35.0*math.pi/180.0
theta_3_guess = -95.0*math.pi/180.0
# q3_guess = 100.0*math.pi/180.0
l_b_guess = 4.0

sol = sp.nsolve(fh.xreplace(repl), (theta_2, theta_3, l_b), (theta_2_guess, theta_3_guess, l_b_guess))
sol

⎡0.162352098936357⎤
⎢                 ⎥
⎢-1.40844422785854⎥
⎢                 ⎥
⎣3.01194235830045 ⎦

In [428]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

points = pd.DataFrame(columns=['1', '2', '3', '4'])
solutions = pd.DataFrame(columns=['theta_2', 'theta_3', 'l_b'])

theta_1_range = np.linspace(0, 2*np.pi, 9)
for theta_1_val in theta_1_range:
    repl[theta_1] = theta_1_val
    sols = sp.nsolve(fh.xreplace(repl), (theta_2, theta_3, l_b), (theta_2_guess, theta_3_guess, l_b_guess))
    theta_2_sol = float(sols[0])
    theta_3_sol = float(sols[1])
    l_b_sol = float(sols[2])

    points = pd.concat([points, pd.DataFrame({
        '1': [np.array(P1.pos_from(P1).to_matrix(N).xreplace(repl).subs({theta_2: theta_2_sol, theta_3: theta_3_sol, l_b: l_b_sol})[:2]).astype(np.float64)],
        '2': [np.array(P2.pos_from(P1).to_matrix(N).xreplace(repl).subs({theta_2: theta_2_sol, theta_3: theta_3_sol, l_b: l_b_sol})[:2]).astype(np.float64)],
        '3': [np.array(P3.pos_from(P1).to_matrix(N).xreplace(repl).subs({theta_2: theta_2_sol, theta_3: theta_3_sol, l_b: l_b_sol})[:2]).astype(np.float64)],
        '4': [np.array(P4.pos_from(P1).to_matrix(N).xreplace(repl).subs({theta_2: theta_2_sol, theta_3: theta_3_sol, l_b: l_b_sol})[:2]).astype(np.float64)],
    })], ignore_index=True)
    solutions = pd.concat([solutions, pd.DataFrame({
        'theta_2': [theta_2_sol],
        'theta_3': [theta_3_sol],
        'l_b': [l_b_sol],
    })], ignore_index=True)

fig = go.Figure()

for i, point in points.iterrows():
    fig.add_shape(
        type='line',
        x0=point['1'][0],
        y0=point['1'][1],
        x1=point['2'][0],
        y1=point['2'][1],
        line=dict(color='blue', width=2),
    )
    fig.add_shape(
        type='line',
        x0=point['2'][0],
        y0=point['2'][1],
        x1=point['3'][0],
        y1=point['3'][1],
        line=dict(color='red', width=2),
    )
    fig.add_shape(
        type='line',
        x0=point['3'][0],
        y0=point['3'][1],
        x1=point['4'][0],
        y1=point['4'][1],
        line=dict(color='green', width=2),
    )
    
    l_b_sol = solutions.iloc[i]['l_b']
    theta_3_sol = solutions.iloc[i]['theta_3']
    output_shaft_rot = float(l_b_sol/repl[l_c] - theta_3_sol - np.pi/4)
    fig.add_shape(
        type='line',
        x0=point['4'][0],
        y0=point['4'][1],
        x1=point['4'][0] + repl[l_c]*np.cos(output_shaft_rot),
        y1=point['4'][1] + repl[l_c]*np.sin(output_shaft_rot),
        line=dict(color='orange', width=2),
    )

fig.update_layout(
    # height=800,
    yaxis_scaleanchor="x",
)
fig.show()

C:\Users\phcre\AppData\Local\Temp\ipykernel_25304\1216961916.py:22: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



## Velocity

In [429]:
P1.set_vel(N, 0)
P4.vel(N)

l_c⋅θ₃̇ c_y + l_b⋅θ₂̇ b_y + lₐ⋅θ₁̇ a_y

In [430]:
# https://moorepants.github.io/learn-multibody-dynamics/configuration.html
t = me.dynamicsymbols._t
fhd = fh.diff(t)
fhd

⎡-lₐ⋅sin(θ₁)⋅θ₁̇ - l_b⋅sin(θ₂)⋅θ₂̇ - l_c⋅sin(θ₃)⋅θ₃̇⎤
⎢                                                ⎥
⎢lₐ⋅cos(θ₁)⋅θ₁̇ + l_b⋅cos(θ₂)⋅θ₂̇ + l_c⋅cos(θ₃)⋅θ₃̇ ⎥
⎢                                                ⎥
⎣            -(θ₂̇ - θ₃̇)⋅sin(θ₂ - θ₃)             ⎦

In [431]:
x = sp.Matrix([theta_2.diff(t), theta_3.diff(t)])
x

⎡θ₂̇⎤
⎢  ⎥
⎣θ₃̇⎦

In [432]:
Amatrix = fhd.jacobian(x)
Amatrix

⎡-l_b⋅sin(θ₂)   -l_c⋅sin(θ₃)⎤
⎢                           ⎥
⎢ l_b⋅cos(θ₂)   l_c⋅cos(θ₃) ⎥
⎢                           ⎥
⎣-sin(θ₂ - θ₃)  sin(θ₂ - θ₃)⎦

In [433]:
b = -fhd.xreplace({theta_2.diff(t): 0, theta_3.diff(t): 0})
b

⎡lₐ⋅sin(θ₁)⋅θ₁̇ ⎤
⎢              ⎥
⎢-lₐ⋅cos(θ₁)⋅θ₁̇⎥
⎢              ⎥
⎣      0       ⎦

In [434]:
x_sol = sp.simplify(Amatrix.LUsolve(b))
x_sol

ValueError: The system is inconsistent.

In [240]:
P4.vel(N).free_dynamicsymbols(N)

{θ₁, θ₂, θ₃, θ₁̇, θ₂̇, θ₃̇}

In [241]:
qd_dep_repl = {
  theta_2.diff(t): x_sol[0, 0],
  theta_3.diff(t): x_sol[1, 0],
  # omega_2: x_sol[0, 0],
  # omega_3: x_sol[1, 0],
}
qd_dep_repl

⎧    -lₐ⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃))⋅θ₁̇       lₐ⋅(2⋅l_b⋅sin(θ₁ - θ₂) - 2⋅l_c⋅ ↪
⎨θ₂̇: ────────────────────────────────────, θ₃̇: ─────────────────────────────── ↪
⎩           l_b⋅sin(sin(θ₂ - θ₃))                                              ↪

↪ sin(-θ₁ + θ₂ + sin(θ₂ - θ₃)) - l_c⋅sin(-θ₁ + θ₃ + sin(θ₂ - θ₃)) + l_c⋅sin(θ₁ ↪
↪ ──────────────────────────────────────────────────────────────────────────── ↪
↪     l_b⋅l_c⋅(sin(-θ₂ + θ₃ + sin(θ₂ - θ₃)) + sin(θ₂ - θ₃ + sin(θ₂ - θ₃)))     ↪

↪  - 2⋅θ₂ + θ₃ - sin(θ₂ - θ₃)))⋅θ₁̇⎫
↪ ────────────────────────────────⎬
↪                                 ⎭

In [242]:
P4.vel(N).xreplace(qd_dep_repl)

lₐ⋅(2⋅l_b⋅sin(θ₁ - θ₂) - 2⋅l_c⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃)) - l_c⋅sin(-θ₁ + θ₃ ↪
────────────────────────────────────────────────────────────────────────────── ↪
                                     l_b⋅(sin(-θ₂ + θ₃ + sin(θ₂ - θ₃)) + sin(θ ↪

↪  + sin(θ₂ - θ₃)) + l_c⋅sin(θ₁ - 2⋅θ₂ + θ₃ - sin(θ₂ - θ₃)))⋅θ₁̇       -lₐ⋅sin( ↪
↪ ───────────────────────────────────────────────────────────── c_y + ──────── ↪
↪ ₂ - θ₃ + sin(θ₂ - θ₃)))                                                      ↪

↪ -θ₁ + θ₂ + sin(θ₂ - θ₃))⋅θ₁̇                 
↪ ──────────────────────────── b_y + lₐ⋅θ₁̇ a_y
↪  sin(sin(θ₂ - θ₃))                          

In [243]:
P4.vel(N).to_matrix(N).xreplace(qd_dep_repl)

⎡ lₐ⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃))⋅sin(θ₂)⋅θ₁̇                   lₐ⋅(-sin(sin(θ₂ ↪
⎢ ────────────────────────────────────────── - lₐ⋅sin(θ₁)⋅θ₁̇ + ─────────────── ↪
⎢             sin(sin(θ₂ - θ₃))                                                ↪
⎢                                                                              ↪
⎢  lₐ⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃))⋅cos(θ₂)⋅θ₁̇                   lₐ⋅(-sin(sin(θ ↪
⎢- ────────────────────────────────────────── + lₐ⋅cos(θ₁)⋅θ₁̇ + ────────────── ↪
⎢              sin(sin(θ₂ - θ₃))                                               ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪ )⋅cos(θ₃) - sin(θ₃)⋅cos(θ₂))⋅cos(θ₂) - sin(θ₂)⋅cos(sin(θ₂)⋅cos(θ₃) - sin(θ₃) ↪
↪ ──────────────────────────────────────────────────────────────────────────── ↪
↪                                                                         l_b⋅ ↪
↪                      

In [244]:
P4.vel(N).xreplace(qd_dep_repl).free_dynamicsymbols(N)

{θ₁, θ₂, θ₃, θ₁̇}

### Solve

In [245]:
fhdt = sp.Matrix(P4.vel(N).xreplace(qd_dep_repl).to_matrix(N)[:2])
fhd = sp.Matrix([fhx, fhy, fhdt])
# fh = sp.trigsimp(fh) # warning, this simplification is slow and may not be necessary
fhd

⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢ lₐ⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃))⋅sin(θ₂)⋅θ₁̇                   lₐ⋅(-sin(sin(θ₂ ↪
⎢ ────────────────────────────────────────── - lₐ⋅sin(θ₁)⋅θ₁̇ + ─────────────── ↪
⎢             sin(sin(θ₂ - θ₃))                                                ↪
⎢                                                                              ↪
⎢  lₐ⋅sin(-θ₁ + θ₂ + sin(θ₂ - θ₃))⋅cos(θ₂)⋅θ₁̇                   lₐ⋅(-sin(sin(θ ↪
⎢- ────────────────────────────────────────── + lₐ⋅cos(θ₁)⋅θ₁̇ + ────────────── ↪
⎣              sin(sin(θ₂ - θ₃))                                               ↪

↪                                             lₐ⋅cos(θ₁) + l_b⋅cos(θ₂) + l_c⋅c ↪
↪                      

In [246]:
import math  # provides pi as a float

repl = {
    l_a: 1.0,
    l_b: 4.0,
    l_c: 3.0,
    l_n: 5.0,
    theta_1: 30.0/180.0*math.pi,  # 30 degrees in radians
    theta_1.diff(t): 1.0,
    # omega_1: 1.0,
    # omega_2: 2.0,
}
repl

{lₐ: 1.0, l_b: 4.0, l_c: 3.0, lₙ: 5.0, θ₁: 0.5235987755982988, θ₁̇: 1.0}

In [247]:
fhd.xreplace(repl)

⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢       0.25⋅(-sin(sin(θ₂)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₂))⋅cos(θ₂) - sin(θ₂)⋅cos(si ↪
⎢       ────────────────────────────────────────────────────────────────────── ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢0.25⋅(-sin(sin(θ₂)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₂))⋅sin(θ₂) + cos(sin(θ₂)⋅cos(θ₃) - ↪
⎢───────────────────────────────────────────────────────────────────────────── ↪
⎣                                                                              ↪

↪                                                                              ↪
↪                          

In [ ]:
theta_2_guess = -75.0/180.0*math.pi  # -75 degrees in radians
theta_3_guess = 100.0/180.0*math.pi  # 100 degrees in radians
# omega_3_guess = 1.0
theta_1_diff_guess = 1.0

sol = sp.nsolve(fhd.xreplace(repl), (theta_2, theta_3, theta_1.diff(t)), (theta_2_guess, theta_3_guess, theta_1_diff_guess))
sol/math.pi*180.0  # to degrees

ValueError: matrix is not positive-definite